In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Function to load CSV files into pandas DataFrames
def load_data(file_paths):
    dataframes = [pd.read_csv(file) for file in file_paths]
    return dataframes

In [ ]:
# Function to preprocess data
def preprocess_data(dataframes):
    # Assuming the last column is the target variable
    X = pd.concat([df.iloc[:, :-1] for df in dataframes], axis=0)
    y = pd.concat([df.iloc[:, -1] for df in dataframes], axis=0)

    # Standardize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    return X, y

In [ ]:
# Define the model
def create_model(input_shape):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
# Function to train models
def train_models(file_paths):
    # Load CSV files
    dataframes = load_data(list(file_paths.values()))

    # Preprocess data
    X_dict = {}
    y_dict = {}
    for label, path in file_paths.items():
        X_dict[label], y_dict[label] = preprocess_data([dataframes[i] for i, p in enumerate(file_paths.values()) if p == path])

    # Split data into train and test sets for each type
    X_train_dict = {}
    X_test_dict = {}
    y_train_dict = {}
    y_test_dict = {}
    for label in file_paths.keys():
        X_train_dict[label], X_test_dict[label], y_train_dict[label], y_test_dict[label] = train_test_split(X_dict[label], y_dict[label], test_size=0.2, random_state=42)

    # Train and evaluate a model for each type
    types = ["Cycler", "Jitter", "Stable", "Stagger"]
    models = {}
    accuracies = {}
    for label in types:
        print(f"Training Model for Type: {label}")
        input_shape = (X_train_dict[label].shape[1],)
        model = create_model(input_shape)
        model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])
        model.fit(X_train_dict[label], y_train_dict[label], epochs=10, batch_size=32, validation_data=(X_test_dict[label], y_test_dict[label]))
        models[label] = model

        test_loss, test_acc = model.evaluate(X_test_dict[label], y_test_dict[label])
        accuracies[label] = test_acc
        print(f"Accuracy for Type {label}: {test_acc}")

    print("Overall Accuracy:")
    print(f"Mean Accuracy: {np.mean(list(accuracies.values()))}")
    print(f"Accuracy for Each Type: {accuracies}")

    return models

In [ ]:
# Function to test a single file with all four models
def test_single_file(file_path, models):
    # Load test data
    test_data = pd.read_csv(file_path)
    X_test, y_test = preprocess_data([test_data])

    # Evaluate each model on the test data
    results = {}
    for label, model in models.items():
        print(f"Evaluating Model for Type: {label}")
        test_loss, test_acc = model.evaluate(X_test, y_test)
        results[label] = test_acc
        print(f"Accuracy for Type {label}: {test_acc}")

    return results

# File paths for training
train_file_paths = {
    "Cycler": "cycler_data.csv",
    "Jitter": "jitter_data.csv",
    "Stable": "stable_data.csv",
    "Stagger": "stagger_data.csv"
}

# File path for testing
test_file_path = "test_data.csv"

# Train models
models = train_models(train_file_paths)

# Test single file with all four models
results = test_single_file(test_file_path, models)
print("Accuracy for Each Type:")
print(results)